# Measuring and comparing the response latencies of models served by custom and Amazon SageMaker pre-built frameworks

---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook.

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/sagemaker_endpoints|latency_testing|endpoint_testing.ipynb)

---

## Background

This notebook will demonstrate how the inference latency of the same model varies when one model is served with [Python Flask](https://flask.palletsprojects.com/en/2.3.x/), [FastAPI](https://fastapi.tiangolo.com/) and [SageMaker pre-built container](https://docs.aws.amazon.com/sagemaker/latest/dg/pre-built-containers-frameworks-deep-learning.html). In all cases, we use a [decision tree](https://scikit-learn.org/stable/modules/tree.html) algorithm from [scikit-learn](https://scikit-learn.org/stable/) package which is trained on the famous [iris dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set). 

The scripts and this notebook can be used to load test and compare latency for any endpoint that you deploy on SageMaker. You can either test a [pre-built SageMaker container](https://docs.aws.amazon.com/sagemaker/latest/dg/docker-containers-prebuilt.html) or [bring your own container](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-byoc-containers.html). [This GitHub repo](https://github.com/aws/amazon-sagemaker-examples/tree/main/advanced_functionality/scikit_bring_your_own) provides a more detailed example on how to bring your own container to SageMaker

## Pre-requisities

### Permissions

Running this notebook requires `SageMakerFullAccess` permissions. Additionally, you will need permissions to publish to Amazon ECR. You can achieve this by adding the managed policy `AmazonEC2ContainerRegistryFullAccess` to the role that you used to start your notebook instance. There's no need to restart your notebook instance when you do this, the new permissions will be available immediately.

### Python Version
This notebook example was run using python 3.10. If you are using SageMaker notebook instance to execute this notebook, please use `Amazon Linux 2, Jupyter Lab 3(notebook-al2-v2)` platform identifier.

### Docker

Docker is required to build and push model images to Amazon ECR

## The experiment

Load testing is carried out on `ml.c5d.18xlarge` SageMaker Notebook Instance type. It is recommended to use a large instance type for load testing in order to be able to provide the adequate load on the Inference Endpoints

![Diagram](img/diagram.png)

# Part 1: Bundling and Pushing Models to Amazon ECR

There is a script inside each model folder to build and push a model image to ECR. The script logs in to ECR, builds and tags an image and pushes the image to ECR

In [ ]:
import concurrent.futures
import subprocess
import re
import os
import pandas as pd

import boto3

from sagemaker import get_execution_role
import sagemaker as sage
from sagemaker.serializers import CSVSerializer
from sagemaker.sklearn.estimator import SKLearn

In [ ]:
models = ["flask-model", "fastapi-model"]

In [ ]:
!chmod +x ./build_and_push.sh

In [ ]:
def build_and_push(model):
    try:
        result = subprocess.run(["./build_and_push.sh", f"{model}"], capture_output=True, text=True)
        if result.returncode == 0:
            print(f"Successfully built and pushed {model} to Amazon ECR")
        else:
            print(f"Error building and pushing {model}")
            print(f"stdout: {result.stdout}")
            print(f"stderr: {result.stderr}")
    except Exception as e:
        print("Exception:", str(e))

Below is the script for building and pushing our custom algorithm containers. It looks for an ECR repository in the account you're using and the current default region (if you're using a SageMaker notebook instance, this will be the region where the notebook instance was created). If the repository doesn't exist, the script will create it, build the image and push to the repository

In [ ]:
!cat ./build_and_push.sh

The cell below executes the scrip and may take a few minutes to run (docker images are building here)

In [ ]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for model in models:
        futures.append(executor.submit(build_and_push, model=model))

    for future in concurrent.futures.as_completed(futures):
        print(future.result())

# Part 2: Training and Deploying models to SageMaker Inference Endpoints

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.


![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/sagemaker_endpoints|latency_testing|endpoint_testing.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/sagemaker_endpoints|latency_testing|endpoint_testing.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/sagemaker_endpoints|latency_testing|endpoint_testing.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/sagemaker_endpoints|latency_testing|endpoint_testing.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/sagemaker_endpoints|latency_testing|endpoint_testing.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/sagemaker_endpoints|latency_testing|endpoint_testing.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/sagemaker_endpoints|latency_testing|endpoint_testing.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/sagemaker_endpoints|latency_testing|endpoint_testing.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/sagemaker_endpoints|latency_testing|endpoint_testing.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/sagemaker_endpoints|latency_testing|endpoint_testing.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/sagemaker_endpoints|latency_testing|endpoint_testing.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/sagemaker_endpoints|latency_testing|endpoint_testing.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/sagemaker_endpoints|latency_testing|endpoint_testing.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/sagemaker_endpoints|latency_testing|endpoint_testing.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/sagemaker_endpoints|latency_testing|endpoint_testing.ipynb)
